In [1]:
import pandas as pd

data = pd.read_csv('data_sample_three_years.csv')

data = data.iloc[:, [0,1,2,3,4]]
data.rename(columns={'date': 'open', 'low':'close', 'open':'high', 'high':'low'}, inplace=True)

data['date'] = pd.to_datetime(data['timestamp'], unit='s')
data = data.iloc[:, [0,5,1,2,3,4]]

data_backup = data.loc[ data.timestamp >= 1675814400 ]
data = data.loc[ data.timestamp < 1675814400 ]

from talib import RSI
data['rsi_14'] = RSI(data['close'], timeperiod=14)
data['rsi_14'] = data['rsi_14'].shift(1)

from talib import MA, SMA, EMA, WMA
data['ma_9'] = MA(data['close'], timeperiod=9)
data['ma_9'] = data['ma_9'].shift(1)
data['sma_9'] = SMA(data['close'], timeperiod=9)
data['sma_9'] = data['sma_9'].shift(1)
data['wma_9'] = WMA(data['close'], timeperiod=9)
data['wma_9'] = data['wma_9'].shift(1)

from talib import MACD
data['macd'], data['signal'], data['hist'] = MACD(data['close'])
data['macd'] = data['macd'].shift(1)
data['signal'] = data['signal'].shift(1)
data['hist'] = data['hist'].shift(1)

from talib import ADX
data['adx'] = ADX(data['high'], data['low'], data['close'])
data['adx'] = data['adx'].shift(1)

from talib import ATR
data['atr'] = ATR(high=data['high'], low=data['low'], close=data['close'], timeperiod=14)
data['atr'] = data['atr'].shift(1)

from talib import SAR
data['sar'] = SAR(high=data['high'], low=data['low'], acceleration=0.02, maximum=0.2)
data['sar'] = data['sar'].shift(1)

from talib import TEMA
data['tema'] = TEMA(data['close'], timeperiod=14)
data['tema'] = data['tema'].shift(1)

from talib import ROC
data['roc'] = ROC(data['close'], timeperiod=14)
data['roc'] = data['roc'].shift(1)

data.dropna(axis=0, inplace=True)

In [2]:
data.shape

(16222, 18)

In [3]:
data.columns[5:]

Index(['close', 'rsi_14', 'ma_9', 'sma_9', 'wma_9', 'macd', 'signal', 'hist',
       'adx', 'atr', 'sar', 'tema', 'roc'],
      dtype='object')

In [4]:
inputs = data[list(data.columns[6:])]
target = data.iloc[:, [5]]

In [5]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=1, shuffle=False)

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
def create_input_sequences(data, seq_length):
    seq = []
    for i in range(len(data) - seq_length):
        seq.append(data[i:i+seq_length])
    return np.array(seq)

In [8]:
X_train_seq = create_input_sequences(X_train_scaled, 10)
y_train_seq = create_input_sequences(y_train.values, 10)

In [9]:
#X_train_reshaped = X_train.values.reshape(X_train.values.shape[0], 1, 12)
#y_train_reshaped = y_train.values.reshape(y_train.values.shape[0], 1, 1)

X_train_reshaped = X_train_seq.reshape(X_train_seq.shape[0], 10, 12)
y_train_reshaped = y_train_seq.reshape(y_train_seq.shape[0], 10, 1)

In [21]:
X_train_reshaped.shape

(12967, 10, 12)

In [11]:
y_train_reshaped.shape

(12967, 10, 1)

In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from keras.regularizers import l2

2024-03-19 07:38:48.397827: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-19 07:38:48.485932: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 07:38:48.486012: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 07:38:48.489940: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 07:38:48.507401: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-19 07:38:50.146023: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [20]:
# Define LSTM model
model = Sequential([
    LSTM(50, input_shape=(12, 10), activation='tanh', kernel_regularizer=l2(0.01)),
    #Dense(64, activation='softplus'),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train_reshaped, epochs=100, batch_size=32)

Epoch 1/100


ValueError: in user code:

    File "/home/bugra/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/bugra/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/bugra/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/bugra/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/home/bugra/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/bugra/.local/lib/python3.10/site-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibili

In [14]:
X_test_seq = create_input_sequences(X_test_scaled, 10)
X_test_reshaped = X_test_seq.reshape(X_test_seq.shape[0], 10, 12)

In [15]:
X_test_reshaped

array([[[0.36764502, 0.20925812, 0.20925812, ..., 0.22726951,
         0.21238729, 0.4816376 ],
        [0.46524768, 0.20864155, 0.20864155, ..., 0.22657958,
         0.21144465, 0.49135174],
        [0.58494364, 0.20877558, 0.20877558, ..., 0.22591725,
         0.21259629, 0.51531157],
        ...,
        [0.45619403, 0.20362222, 0.20362222, ..., 0.22176508,
         0.20629335, 0.48552481],
        [0.57041044, 0.20359801, 0.20359801, ..., 0.22091028,
         0.20832391, 0.49927018],
        [0.54564529, 0.20420363, 0.20420363, ..., 0.20751844,
         0.20949598, 0.49267777]],

       [[0.46524768, 0.20864155, 0.20864155, ..., 0.22657958,
         0.21144465, 0.49135174],
        [0.58494364, 0.20877558, 0.20877558, ..., 0.22591725,
         0.21259629, 0.51531157],
        [0.43491636, 0.20785116, 0.20785116, ..., 0.22528141,
         0.21085235, 0.4851734 ],
        ...,
        [0.57041044, 0.20359801, 0.20359801, ..., 0.22091028,
         0.20832391, 0.49927018],
        [0.5

In [16]:
y_pred = model.predict(X_test_reshaped)

102/102 [==============================] - 2s 5ms/step


In [17]:
y_test

,close
13017,1744.47
13018,1765.01
13019,1735.29
13020,1725.50
13021,1725.71
...,...
16257,1638.52
16258,1656.42
16259,1664.93
16260,1667.43


In [18]:
y_pred

array([[2691.556],
       [2691.556],
       [2691.556],
       ...,
       [2691.556],
       [2691.556],
       [2691.556]], dtype=float32)